#<u>Titre du projet : Optimisation den la composition de portefeuille d'actifs</u>

### <u>Membres du projet</u>:
  1. **BADOLO** Christian Thomas : Implementaton du gradient projété et programmation fonctionnelle avec Scipy
  2. **NIAMPA** Soumaïla : Modelisation mathématique et étude théorique
  3. **SEBEOGO** Landry Yves Joël : Etude théorique et implementaton du gradient projété
  4. **YEBOUA** Franck : Implementaton du gradient et discussion
  5.**ZONGO** Sosthène : implémentation de l'agorithme d'uzawa et extration des données Yahoo finance


# <u>Formulation mathématique sous forme d'un problème d'optimisation bien posé</u>


* Considérons un investisseur qui souhaite allouer du capital parmi $N$ titres.
 Notons $p_i(0)$ le prix(connu) investi dans l’actif $i$ à $t=0$ et $p_i(r)$ le prix(aléatoire) investi dans l’actif $i$ à $t=r$

* Modélisons par la variable aléatoire le taux $y_i$ de rendement du titre $i$ sur la période $r$ par $y_i = \dfrac{pᵢ(r)}{pᵢ(0)}$

* Notons $\mu_i=  \mathbb{E}[y_i] $ sa Valeur attendue. L'investisseur, ne voulant faire profil, cherche à maximiser le rendement de l'investissement tout en essayant de maintenir son risque à un niveau acceptable relativement bas.
Notons $X$ le portefeuille où $x_i$ représente la proportion du capital allouer à l’actif $i$ à $t=0$ (fraction des fonds investis dans le titre $i$).
Nous pouvons donc exprimer le rendement aléatoire du portefeuille par \begin{align*}
R &=\sum_{i=1}^N x_iy_i=X^TY \\
\end{align*}

* Le rendement attendu du portefeuille $\mu_X =\mathbb{E}[R] = X^T\mathbb{E}[Y] = X^T\mu$.
* La variance du portefeuille $\sigma²_X =\mathbb{Var}[R]= \mathbb{Var}[X^TR]= X^T\Omega X$.


* Problème d'optimisation

  L'investisseur a pour objectif de maximiser son rendrment $\mathbb{E}[R]$ tout en maintenant $\sigma²_X $ à un seuil acceptable.

$$
\max_{X\in\mathbb{R}^N} X^T {\mu}\quad \text{sous}\quad X^T \Omega X \leq \sigma^2
$$
* Problème sous forme standard
$$
\min_{X\in\mathbb{R}^N} -X^T {\mu}\quad \text{sous}\quad X^T \Omega X \leq \sigma^2
$$

**La fonction coût définie de l'ensemble $\mathbb{R}^N$ vers $\mathbb{R}$ est $J_0(X) = -X^T{\mu}\quad$ et
l'ensemble des contraintes est $J_1(X) = X^TΩX - \sigma^2 \leq 0$, qui représente la contrainte de risque.**

L'on ajoute également les contraintes $\sum_{i} X_i = 1 \quad \text{et} \quad X_i \geq 0$


# <u>Analyse mathématique du problème d'optimisation </u>

## <u>Existence et unicité</u>

* $J_0(X) = -X^T{\mu}\quad$ est linéaire donc convexe.
* L'ensemble $K = \{X \in \mathbb{R}^N \mid X^T\Omega X - \sigma^2 \leq 0\}$  est compact
* $J_0$ est continue donc admet un minimun. De plus, $K$ et $J_0$ étant convexes l'unicité du minimum est assuré

## <u>Caractértisation du minimum</u>

A partir des conditions d'optimalité du lagrangien associé au problème on arrive à écire les conditions dites de **karush-Kuhn-Tucker**.
* $-{\mu} + 2λΩ\bar{X} = 0$
* $\bar{λ}\geq 0$
* $\bar{X}^T\Omega\bar{X} - \sigma^2 \leq 0$
* $\bar{λ}(\bar{X}^TΩ\bar{X} - σ^2) = 0$

## <u>Sélection d'un algorithme de résolution</u>


Le problème d'optimisation étant sous contraintes, l'on a le choix entre la méthode du gradient projété et l'algorithme d'Uzawa.

###Gradient projété

**1. Initialisation:** $k=0:$ choix de $x_0$ et de $\rho_0>0$

**2. Iteration:**   $x_{k+1}=\pi_K\left(x_k-\rho_k \nabla J\left(x_k\right)\right)$;

**3. Critère d'arrêt:** Si $\left\|x_{k+1}-x_k\right\|<\varepsilon$, STOP

La projection $\pi_K$a  été déterminée par la resolution du problème $min \dfrac{1}{2}||x-y||^2$ avec $x \in K$ et $y \in R^N$

###Algorithme d'uzawa

On le défini comme suit:

**1. Initialisation**:  $k = 0$, choisir $\lambda^{(0)} \in \mathbb{R}^p$ et $\mu^{(0)} \in \mathbb{R}^q_+$

Tant que le critère d'arrêt n'est pas satisfait:

**2. Calculer** $x^{(k)} \in \mathbb{R}^n$ solution de $\min_{x \in \mathbb{R}^n} L(x, \lambda^{(k)}, \mu^{(k)})$

**3. Calculer** $\lambda^{(k+1)}$ et $\mu^{(k+1)}$:

\begin{cases}
\lambda^{(k+1)}_i = \lambda^{(k)}_i + \rho h_i(x^{(k)}), & i = 1, \ldots, p \\
\mu^{(k+1)}_j = \max\{0, \mu^{(k)}_j + \rho g_j(x^{(k)})\}, & j = 1, \ldots, q
\end{cases}





* Convergence

* Vitesse de convergence

# Extration des Données sur Yahoo Finance

In [ ]:
import time
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
from numpy.linalg import norm

In [ ]:
def data(code_tickers):
    # Récupération des données des tickers à l'aide de l'API de Yahoo Finance
    data_frames, n_days_back = [], 365
    for code in code_tickers:
        stock = yf.Ticker(code)
        df = stock.history(period=f"{n_days_back}d")
        df = df.reset_index()
        df = df[["Date", "Close"]]
        df.columns = ["Date", code]
        data_frames.append(df)

    market_data_frame = data_frames[0]
    for df in data_frames[1:]:
        market_data_frame = pd.merge(market_data_frame, df, on="Date", how="outer")

    market_data_frame = market_data_frame.set_index("Date")
    return market_data_frame

code_tickers = ["TSLA", "GOOG","AMZN","META","AAPL","TNET","BRZE"]
market_data_frame = data(code_tickers) #Table contenant les données de tesla, amazon, meta, netflix, apple, aurora et trinet

mu = np.array(market_data_frame .mean()) #calcul de la moyenne mu
omega = np.array(market_data_frame .cov()) #calcul de la covariance

print(mu)
print(omega)
#print(market_data_frame. cov())

[236.76465747 111.83336309 118.78089602 186.6723016  155.34749827
  81.51569888  36.71476717]
[[3913.9413354   738.5027041  1284.79747823 1435.90537105  436.68753615
   302.95845541  440.73616178]
 [ 738.5027041   235.58416028  321.02530988  604.71956704  150.489609
   103.72852804  105.93674207]
 [1284.79747823  321.02530988  522.85398767  699.87678128  201.78155264
   139.47774177  169.71879369]
 [1435.90537105  604.71956704  699.87678128 2784.16951733  544.68519332
   357.50118375  282.95577504]
 [ 436.68753615  150.489609    201.78155264  544.68519332  187.15821202
   103.81280843   65.893672  ]
 [ 302.95845541  103.72852804  139.47774177  357.50118375  103.81280843
    81.05521579   41.4744002 ]
 [ 440.73616178  105.93674207  169.71879369  282.95577504   65.893672
    41.4744002    83.30150443]]


# Resolution avec scipy

In [ ]:
def objective(x,mu):
    return -mu @ x  # Minimiser le négatif du rendement attendu

def optimize_portfolio(alpha):
    contraintes= ({'type': 'eq', 'fun': lambda x: sum(x) - 1},{'type': 'ineq', 'fun': lambda x: alpha**2 - x @ omega @ x})
    bounds = [(0, None)] * len(mu)  # Proportions positives
    x0 = np.ones(len(mu)) / len(mu) # Répartition initiale égale
    result = minimize(objective, x0, args = (mu,), method = 'SLSQP', bounds = bounds, constraints = contraintes) #resolution du problème d'optimisation
    return  result.x

alpha = 0.01  # Exemple de niveau de risque acceptable
debut = time.time()
proportions_scipy = optimize_portfolio(alpha)
fin= time.time()
t1 = fin - debut
print("Proportions optimales :", proportions_scipy)

Proportions optimales : [9.50006416e-12 2.84932056e-14 5.76545445e-14 1.50044900e-11
 2.80437847e-12 5.15497625e-01 4.84502996e-01]
Valeur du portefeuille:  59.80956388333877


# Gradient projété

In [ ]:
#Definition de la projection
def J(x,y):
    return 0.5*norm(x - y)**2

def projection(y, omega, alpha):
    contraintes= ({'type': 'eq', 'fun': lambda x: sum(x) - 1},{'type': 'ineq', 'fun': lambda x: alpha**2 - x@omega@x})
    bounds = [(0, None)] * len(mu)  # Proportions positives
    x0 = np.ones(len(mu)) / len(mu) # Répartition initiale égale
    result = minimize(J, x0,args = (y,), method = 'SLSQP', bounds = bounds, constraints =  contraintes) #resolution du problème d'optimisation
    return  result.x

In [ ]:
# Test projection sur K
y_0 = np.ones(len(mu))
#y_0 = np.array([237.06259348, 118.89525767])
y = projection(y_0, omega, 0.02)
print(y)

[7.88027660e-14 0.00000000e+00 7.29425893e-12 0.00000000e+00
 0.00000000e+00 5.14268027e-01 4.86298911e-01]


In [ ]:
# Définiton de la fonction objective et son gradient analytique
def J0(mu,x):
    return np.dot(-x,mu)
def GradJ0(mu,x):
    return -mu

In [ ]:
#Test fonction coût et son gradient
x = np.array([2, 1, 5, 2])
m = np.array([2, 7, 3, 9])
print("J0 =",J0(m,x))
print("GradJ0 =",GradJ0(m,x))

J0 = -44
GradJ0 = [-2 -7 -3 -9]


In [ ]:
def gradientProjete (mu,omega,x0 ,alpha ,pas=1e-8, tol=1e-6) :
    xk  = x0
    iter = 0
    err = 1
    while err > tol:
      x_old = np.copy(xk)
      xk = projection(xk - np.dot(pas, GradJ0(mu,xk)), omega, alpha)
      err = norm(x_old - xk)
      iter += 1
    return xk
alpha = 0.01
x0 = np.ones(len(mu)) / len(mu) # Répartition initiale égale
debut = time.time()
proportions_gradientProjete = gradientProjete (mu, omega, x0 ,alpha)
fin= time.time()
t2 = fin - debut

In [ ]:
print("Proportions optimales",proportions_gradientProjete)

Proportions optimales [1.66901241e-21 3.29469001e-20 1.88784719e-18 3.79895975e-22
 3.71990649e-22 5.13796496e-01 4.86203504e-01]
Valeur du portefeuille:  59.73332890104629


# Algorithme d'Uzawa

In [ ]:
# Fonction g(x_k)
def g(x_k, omega, alpha):
    return  x_k @ omega @ x_k - alpha**2

# Fonction f(x_k)
def h(x_k):
    return np.sum(x_k) - 1

def uzawa_algorithm(alpha, omega,lamba_k=0, beta_k=1, rho=1e-6, epsilon=1e-4):
    lambda_k, beta_k =  0, 1
    iter = 1
    err = 1
    bounds = [(0, None)] * len(mu)
    x0 = np.ones(len(mu)) / len(mu)
    while err > epsilon:
        x_old = x0
        objective = lambda x : -mu @ x+ lambda_k*h(x) + beta_k*g(x, omega, alpha)
        result =  minimize(objective, x0, bounds = bounds)
        x_k = result.x
        beta_k= max(0, beta_k + rho * g(x_k, omega, alpha))
        lambda_k = lambda_k  + rho * h(x_k)
        err = norm(x_k - x_old)
        x0 = x_k
        iter += 1
    return x_k

alpha = 0.01
debut = time.time()
proportions_Uzawa = uzawa_algorithm(alpha,omega)
fin= time.time()
t3 = fin - debut
print("Proportions optimales",proportions_Uzawa)

Proportions optimales [0.         0.         0.         0.         0.41500313 0.
 0.        ]
Valeur du portefeuille:  64.46969781738812


# Discution et interpretation

In [ ]:
print("Valeur du portefeuille scipy: ", mu @ proportions_scipy)
print("Valeur du portefeuille gradient projété: ", mu @ proportions_gradientProjete)
print("Valeur du portefeuille Uzawa: ", mu @ proportions_Uzawa)

Valeur du portefeuille scipy:  59.80956388333877
Valeur du portefeuille gradient projété:  59.73332890104629
Valeur du portefeuille Uzawa:  64.46969781738812


In [ ]:
print("Temps d'exécution de resolution avec scipy :", t1)
print("Temps d'exécution du gradient projété :", t2)
print("Temps d'exécution de l'algorithme d'Uzawa :", t3)

Temps d'exécution de resolution avec scipy : 0.28118038177490234
Temps d'exécution du gradient projété : 0.33217716217041016
Temps d'exécution de l'algorithme d'Uzawa : 0.05890512466430664


En comparant ces différents résultats, on observe que **l'algorithme d'Uzawa** a produit la meilleure valeur du portefeuille, tandis que l'algorithme du gradient projeté a donné la valeur la plus basse. En termes de temps d'exécution, l'algorithme d'Uzawa est nettement plus rapide que les deux autres méthodes.

Ces résultats soulignent l'importance de choisir l'algorithme approprié en fonction des objectifs spécifiques.